In [1]:
print("test")

test


# **Run prior to starting the server and chatbot**

In [2]:
#Installations for constructing knowledge graph
!pip install llama-index-llms-openai
!pip install llama-index-readers-file
!pip install llama-index-embeddings-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled 

In [3]:
#installations for Fast API
!pip install fastapi nest-asyncio pyngrok uvicorn
!ngrok config add-authtoken $NGROK_AUTHTOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.4 MB/s eta 0:00:00
add-authtoken - save authtoken to configuration file

USAGE:
  ngrok config add-authtoken TOKEN [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080                        # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure yo

In [4]:
# Needed for information extraction
import requests
from bs4 import BeautifulSoup
import networkx as nx
import spacy
from prettytable import PrettyTable
import textwrap
from google.colab import userdata

# For Google custom search engine
api_key = userdata.get('api_key')
cse_id = userdata.get('cse_id')
nlp = spacy.load("en_core_web_sm")

In [5]:
# To store the extracted info in GDrive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api-key')

In [7]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [8]:
# For constructing knowledge graph and displaying results
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.core import StorageContext

In [9]:
import os
# Define the path to the specific folder in Google Drive
folder_path = '/content/drive/My Drive/KG'

# Create the directory if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Define the path for the file
file_path = os.path.join(folder_path, 'results.txt')

In [10]:
# Default value
curr_query = 'use cases of transformers in machine learning'

In [11]:
os.environ['NGROK_AUTHTOKEN'] = userdata.get('ngrok_auth_token')

# **Chatbot calls these functions**

In [ ]:
# Extract information using the Google API key and Custom search engine id
"""def google_search(query, api_key, cse_id, num_results=5):
    search_url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cse_id}&num={num_results}"
    response = requests.get(search_url)
    results = response.json().get('items', [])
    return [item['link'] for item in results]

def extract_relevant_info(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('title').get_text()
        paragraphs = soup.find_all('p')
        text = "\n".join([p.get_text() for p in paragraphs])
        return {"title": title, "text": text}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# Function to wrap text for table display
def wrap_text(text, width):
    return "\n".join(textwrap.wrap(text, width=width))

# Function to format and print the output using PrettyTable
def print_formatted_output(results):
    table = PrettyTable()
    table.field_names = ["Title", "URL", "Text"]

    for result in results:
        wrapped_title = wrap_text(result['title'], width=30)
        wrapped_url = wrap_text(result['url'], width=30)
        wrapped_text = wrap_text(result['text'], width=50)
        table.add_row([wrapped_title, wrapped_url, wrapped_text])

    print(table)



def user_input_kg(query):
  if len(query)>0:
    curr_query = query
  else:
    curr_query = 'use cases of transformers in machine learning'
  query = curr_query
  urls = google_search(query, api_key, cse_id)
  all_entities = set()
  all_relationships = []
  results = []

  for url in urls:
      info = extract_relevant_info(url)
      if "error" not in info:
          info['url'] = url
          results.append(info)
  print_formatted_output(results)
  return results
"""

'def google_search(query, api_key, cse_id, num_results=5):\n    search_url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cse_id}&num={num_results}"\n    response = requests.get(search_url)\n    results = response.json().get(\'items\', [])\n    return [item[\'link\'] for item in results]\n\ndef extract_relevant_info(url):\n    try:\n        response = requests.get(url)\n        response.raise_for_status()\n        soup = BeautifulSoup(response.content, \'html.parser\')\n        title = soup.find(\'title\').get_text()\n        paragraphs = soup.find_all(\'p\')\n        text = "\n".join([p.get_text() for p in paragraphs])\n        return {"title": title, "text": text}\n    except requests.exceptions.RequestException as e:\n        return {"error": str(e)}\n\n# Function to wrap text for table display\ndef wrap_text(text, width):\n    return "\n".join(textwrap.wrap(text, width=width))\n\n# Function to format and print the output using PrettyTable\ndef print_form

In [12]:
# For handling missing text
# Extract information using the Google API key and Custom search engine id
import requests
from bs4 import BeautifulSoup
import spacy
import textwrap
from prettytable import PrettyTable

# Load the spaCy language model
nlp = spacy.load("en_core_web_sm")


def google_search(query, api_key, cse_id, num_results=5):
    search_url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cse_id}&num={num_results}"
    response = requests.get(search_url)
    results = response.json().get('items', [])
    print("Google search completed")
    return [item['link'] for item in results]

def extract_relevant_info(url):
    try:
        # Set a timeout for the request
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        title_tag = soup.find('title')
        if not title_tag:
            print(f"Skipping {url}: No title tag found.")
            return {"error": "No title tag found"}

        title = title_tag.get_text()
        paragraphs = soup.find_all('p')
        text = "\n".join([p.get_text() for p in paragraphs])

        return {"title": title, "text": text}

    except requests.exceptions.RequestException as e:
        print(f"Skipping {url}: {str(e)}")
        return {"error": str(e)}

# Function to wrap text for table display
def wrap_text(text, width):
    return "\n".join(textwrap.wrap(text, width=width))

# Function to format and print the output using PrettyTable
def print_formatted_output(results):
    table = PrettyTable()
    table.field_names = ["Title", "Text"]

    for result in results:
        wrapped_title = wrap_text(result['title'], width=30)
        wrapped_text = wrap_text(result['text'], width=50)
        table.add_row([wrapped_title, wrapped_text])

    print(table)

# Function to save the extracted text and title to a file
def save_to_file(results, filename="output.txt"):
    with open(filename, "w") as file:
        for result in results:
            file.write(result['title'] + "\n\n")
            file.write(result['text'] + "\n")
            file.write("\n" + "="*50 + "\n\n")  # Separator between entries

def user_input_kg(query):
    if len(query) > 0:
        curr_query = query
    else:
        curr_query = 'use cases of transformers in machine learning'
    query = curr_query
    urls = google_search(query, api_key, cse_id)
    all_entities = set()
    all_relationships = []
    results = []

    for url in urls:
        print("Currently processing", url)
        info = extract_relevant_info(url)
        if "error" not in info:
            results.append(info)

    print_formatted_output(results)
    return results


user_input_kg("Altera infuses AI")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Google search completed
Currently processing https://www.allaboutcircuits.com/news/altera-infuses-ai-into-new-mid-range-fpgas/
Skipping https://www.allaboutcircuits.com/news/altera-infuses-ai-into-new-mid-range-fpgas/: 403 Client Error: Forbidden for url: https://www.allaboutcircuits.com/news/altera-infuses-ai-into-new-mid-range-fpgas/
Currently processing https://www.intel.com/content/www/us/en/newsroom/news/intel-altera-bring-ai-to-embedded-world.html
Currently processing https://www.eetimes.com/embedded-world-2024-sandra-rivera-talks-about-altera-and-ai/
Skipping https://www.eetimes.com/embedded-world-2024-sandra-rivera-talks-about-altera-and-ai/: HTTPSConnectionPool(host='www.eetimes.com', port=443): Read timed out. (read timeout=10)
Currently processing https://www.linkedin.com/posts/sandra-rivera-6a24291_women4ew-womenintech-wearealtera-activity-7183884756843999232-ZNMo
Currently processing https://cfotech.in/story/intel-launches-standalone-fpga-company-to-revolutionise-ai
Skippi

[{'title': 'Intel and Altera Announce Edge and FPGA Offerings for AI at Embedded...',
  'text': "You can easily search the entire Intel.com site in several ways.\nYou can also try the quick links below to see results for most popular searches.\nThe browser version you are using is not recommended for this site.Please consider upgrading to the latest version of your browser by clicking one of the following links.\nNew edge-optimized processors and FPGAs bring AI everywhere across edge computing markets including retail, industrial and healthcare.\n\nApril 8, 2024\nContact Intel PR\n\nFollow Intel Newsroom on social:\n\n\n\n\n\n\n\n\n\n\n\nBy\n\nWhat’s New:\xa0Today at Embedded World, Intel and Altera, an Intel Company, announced new edge-optimized processors, FPGAs and programmable market-ready solutions extending powerful AI capabilities into edge computing. These products will power AI-enabled edge devices applicable to industries across retail, healthcare, industrial, automotive, def

In [13]:

# Function to save results to text file in Google Drive
def save_results_to_text(results, filename=file_path):
    try:
        with open(filename, mode='w', encoding='utf-8') as file:
            for result in results:
                file.write(f"Text: {result['text']}\n")
        # Check if the file was created and is not empty
        if os.path.exists(filename) and os.path.getsize(filename) > 0:
            print(f"File '{filename}' created successfully.")
            return True
        else:
            print(f"File '{filename}' not created or is empty.")
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False


In [14]:
# Stores the results in results.txt file in a GDrive folder titled 'KG'
def store_extracted_info(results):

  stored_result=save_results_to_text(results)
  folder_path = '/content/drive/My Drive/KG'
  files = os.listdir(folder_path)
  print(files)
  # Commenting the following line of code because the graph has already been constructed for the demo
  # construct_knowledge_graph()
  run_async_function()
  if stored_result:
    return 200
  else:
    return 400

In [28]:
# Load all the text files in the folder to construct knowledge graph
"""
Reads documents from the KG directory (GDrive) and creates KnowledgeGraphIndex with triplets,
and the embeddings. The graph is stored in SimpleGraphStore
"""

import networkx as nx
import matplotlib.pyplot as plt
def construct_knowledge_graph():
  documents = SimpleDirectoryReader(
    "/content/drive/My Drive/KG"
  ).load_data()
  llm = OpenAI(temperature=0, model="gpt-4")
  Settings.llm = llm
  Settings.chunk_size = 512
  graph_store = SimpleGraphStore()
  storage_context = StorageContext.from_defaults(graph_store=graph_store)
  # Considering embeddings
  global new_index
  new_index = KnowledgeGraphIndex.from_documents(
      documents,
      max_triplets_per_chunk=2,
      include_embeddings=True,
  )
  global query_engine
  query_engine = new_index.as_query_engine(
      include_text=True, response_mode="tree_summarize"
    )
  print("Knowledge graph constructed")
  nodes = set()
  edges = []

  for triplet in new_index.get_triplets():
      subject, predicate, obj = triplet
      nodes.add(subject)
      nodes.add(obj)
      edges.append((subject, obj, predicate))
  G = nx.DiGraph()
  G.add_nodes_from(nodes)
  for edge in edges:
      G.add_edge(edge[0], edge[1], label=edge[2])

  # Draw the graph
  pos = nx.spring_layout(G)
  nx.draw(G, pos, with_labels=True, node_color='lightblue', font_weight='bold', node_size=3000, edge_color='gray')
  edge_labels = nx.get_edge_attributes(G, 'label')
  nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

  plt.show()




In [16]:
# As the knowledge graph construction takes a longer time, and HTTP requests time out shortly, the knowledge graph function is called async
import threading
import time

def run_async_function():
    thread = threading.Thread(target=construct_knowledge_graph)
    thread.start()
    print("Sync function has been called")

In [ ]:
# Do not run now  : Note: This code block is for testing if the responses are received as expected
# Expecting a correct response for this question
query_engine = new_index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)
response = query_engine.query(
    "What is machine learning",
)
display(Markdown(f"{response}"))
# No information about this query will be received as the data doesn't exist in the document
response = query_engine.query(
    "What is Jyothi studying",
)
display(Markdown(f"{response}"))

In [17]:
def fn_query_engine(question):
  global query_engine
  query_engine = new_index.as_query_engine(
      include_text=False, response_mode="tree_summarize"
    )



In [18]:
# The chatbot accesses these APIs
# Creating a FastAPI application
from fastapi import FastAPI,Query
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

# Called when the user logs in to the application and wants to talk to the chatbot
@app.get('/answer_questions')
async def answer_questions(question: str = Query(..., description="The question to be answered")):
    query_engine = new_index.as_query_engine(
      include_text=False, response_mode="tree_summarize"
    )
    response = query_engine.query(
        question,
    )
    print(str(response))
    return {'question': question, 'response': str(response)}

# Get info from the user for the topic to generate a knowledge graph about
@app.get('/generate_knowledge_graph')
async def generate_knowledge_graph(kg_query: str = Query(..., description="Topic to generate the knowledge graph on")):
    print("generate knowledge graph")
    results = user_input_kg(kg_query)
    print(results)
    status = store_extracted_info(results)
    print(status)
    return {'question': kg_query, 'response': str(status)}

In [30]:

if new_index:  print("YES")


YES


In [29]:
# Set up a local web server to make it accessible over the internet, the APIs will be called by the React.js application
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('URL: ', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

URL:  https://5302-34-87-191-170.ngrok-free.app


INFO:     Started server process [233]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


generate knowledge graph
Google search completed
Currently processing https://www.allaboutcircuits.com/news/altera-infuses-ai-into-new-mid-range-fpgas/
Skipping https://www.allaboutcircuits.com/news/altera-infuses-ai-into-new-mid-range-fpgas/: 403 Client Error: Forbidden for url: https://www.allaboutcircuits.com/news/altera-infuses-ai-into-new-mid-range-fpgas/
Currently processing https://www.intel.com/content/www/us/en/newsroom/news/intel-altera-bring-ai-to-embedded-world.html
Currently processing https://www.eetimes.com/embedded-world-2024-sandra-rivera-talks-about-altera-and-ai/
Skipping https://www.eetimes.com/embedded-world-2024-sandra-rivera-talks-about-altera-and-ai/: HTTPSConnectionPool(host='www.eetimes.com', port=443): Read timed out. (read timeout=10)
Currently processing https://www.linkedin.com/posts/sandra-rivera-6a24291_women4ew-womenintech-wearealtera-activity-7183884756843999232-ZNMo
Currently processing https://cfotech.in/story/intel-launches-standalone-fpga-company-t

Exception in thread Thread-18 (construct_knowledge_graph):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-28-c139f9fc0d1f>", line 33, in construct_knowledge_graph
AttributeError: 'KnowledgeGraphIndex' object has no attribute 'get_triplets'


Knowledge graph constructed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [233]


In [31]:
!pip install llama_index networkx matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 16.6 MB/s eta 0:00:00


In [38]:
## create graph
from pyvis.network import Network

g = new_index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("kg_graph.html")


kg_graph.html


In [37]:
!pip install pyvis

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 19.6 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
